# Anomaly Detection For Timeseries of Networks

There is a particular type of sea slug who have gills on the outside of their body. When you squirt water at these gills, they withdraw into the slug. The interesting thing about this type of slug is that the circuitry for this gill-withdrawal circuit has been entirely mapped out, from the neurons which detect and transmit information about the water into the slug's brain, to the neurons that leave the brain and fire at muscles, causing these muscles to pull back.

Say you're a researcher studying these sea slugs, and you have a bunch of brain networks of the same slug. Each node is one of its neurons, and edges denote connections between neurons. Each of the brain networks that you have were taken at different time points: some before water started getting squirted at the slug's gills, and some as the water was getting squirted. Your goal is to reconstruct when water started to get squirted, using only the networks themselves. You hypothesize that there should be some signal change in your networks which can tell you the particular time at which water started getting squirted. Given the network data you have, how do you figure out which timepoints these are?

The broader class of problems this question addresses is called *anomaly detection*. The idea in general is that you have a bunch of snapshots of the same network over time. Although the nodes are the same, the edges are changing at each time point. Your goal is to figure out which time points correspond to the most change, either in the entire network or in particular groups of nodes.

In particular, what we would really like to do is separate the signal from the noise. All of the nodes in the network are likely changing a bit over time, since there is some variability intrinsic in the system. Random noise might just dictate that some edges get randomly deleted and some get randomly created, but we want to figure out when neurons are changing as the result of the squirting in particular.

For instance, let's call the latent positions for our network $X^{(t)}$ when the we've taken a snapshot of the network at time $t$. We can assume that  $||X^{(t)} - X^{(t-1)}|| < c$: in other words, that the difference in norm between the latent positions at time $t$ and the latent positions at time $t-1$ is less than some constant $c$. The problem essentially boils down to finding the times for which $||X^{(t)} - X^{(t-1)}|| > c$.

## Counting Changed Edges

One of the simplest approaches to this problem might just be to figure out which node has the highest count of edge changes across your timeseries. For each node across the timeseries, you'd count the number of new edges that appeared, and the number of existing edges that were deleted. Whichever count is highest is your anomalous node.

This might give you a rough estimate, but it doesn't account for other important pieces of information. You might be interested in which other nodes new edges were formed with, and which nodes edges were deleted from, for instance.

## 

## References



- j1's paper -- heritability
- vivek's paper -- mcc

## Notes

guodong's stuff: uses MASE and OMNI combined with DCORR to do hypothesis testing
- vivek did something similar for MCC